In [ ]:
!pip install pydub -q
!apt-get install ffmpeg -yq

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from google.colab import drive
from pydub import AudioSegment
import os

In [ ]:
#specify folder
base_folder_path = '/content/drive/MyDrive/audio_stemming/Demucs/htdemucs'  # Replace with the path to your main folder
output_folder = '/content/drive/MyDrive/audio_stemming/Demucs_Output'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Step 4: Find and process each "vocals" audio file
file_count = 1
for folder_name in os.listdir(base_folder_path):
    folder_path = os.path.join(base_folder_path, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if "vocals.mp3" == file_name.lower():  # Adjust for case-insensitivity
                file_path = os.path.join(folder_path, file_name)

                # Step 5: Convert audio to .wav
                audio = AudioSegment.from_file(file_path)
                output_filename = f'Demucs_vocal_{file_count}.wav'
                output_path = os.path.join(output_folder, output_filename)

                # Export as .wav
                audio.export(output_path, format="wav")
                print(f"Converted and saved: {output_filename}")

                # Increment file count for naming
                file_count += 1

Converted and saved: Demucs_vocal_1.wav
Converted and saved: Demucs_vocal_2.wav
Converted and saved: Demucs_vocal_3.wav
Converted and saved: Demucs_vocal_4.wav
Converted and saved: Demucs_vocal_5.wav
Converted and saved: Demucs_vocal_6.wav
Converted and saved: Demucs_vocal_7.wav
Converted and saved: Demucs_vocal_8.wav
Converted and saved: Demucs_vocal_9.wav
Converted and saved: Demucs_vocal_10.wav
Converted and saved: Demucs_vocal_11.wav
Converted and saved: Demucs_vocal_12.wav
Converted and saved: Demucs_vocal_13.wav
Converted and saved: Demucs_vocal_14.wav
Converted and saved: Demucs_vocal_15.wav
Converted and saved: Demucs_vocal_16.wav
Converted and saved: Demucs_vocal_17.wav
Converted and saved: Demucs_vocal_18.wav
Converted and saved: Demucs_vocal_19.wav
Converted and saved: Demucs_vocal_20.wav
Converted and saved: Demucs_vocal_21.wav
Converted and saved: Demucs_vocal_22.wav
Converted and saved: Demucs_vocal_23.wav
Converted and saved: Demucs_vocal_24.wav
Converted and saved: Demu

In [ ]:
# Step 4: Find and process each "vocals" audio file
file_count = 1
for file_name in os.listdir(base_folder_path):
    if "vocals" in file_name.lower():  # Adjust for case-insensitivity
        file_path = os.path.join(base_folder_path, file_name)

        # Step 5: Convert audio to .wav
        audio = AudioSegment.from_file(file_path)
        output_filename = f'Wave_vocal_{file_count}.wav'
        output_path = os.path.join(output_folder, output_filename)

        # Export as .wav
        audio.export(output_path, format="wav")
        print(f"Converted and saved: {output_filename}")

        # Increment file count for naming
        file_count += 1

Converted and saved: Wave_vocal_1.wav
Converted and saved: Wave_vocal_2.wav
Converted and saved: Wave_vocal_3.wav
Converted and saved: Wave_vocal_4.wav
Converted and saved: Wave_vocal_5.wav
Converted and saved: Wave_vocal_6.wav
Converted and saved: Wave_vocal_7.wav
Converted and saved: Wave_vocal_8.wav
Converted and saved: Wave_vocal_9.wav
Converted and saved: Wave_vocal_10.wav
Converted and saved: Wave_vocal_11.wav
Converted and saved: Wave_vocal_12.wav
Converted and saved: Wave_vocal_13.wav
Converted and saved: Wave_vocal_14.wav
Converted and saved: Wave_vocal_15.wav
Converted and saved: Wave_vocal_16.wav
Converted and saved: Wave_vocal_17.wav
Converted and saved: Wave_vocal_18.wav
Converted and saved: Wave_vocal_19.wav
Converted and saved: Wave_vocal_20.wav
Converted and saved: Wave_vocal_21.wav
Converted and saved: Wave_vocal_22.wav
Converted and saved: Wave_vocal_23.wav
Converted and saved: Wave_vocal_24.wav
Converted and saved: Wave_vocal_25.wav
Converted and saved: Wave_vocal_26

### VOCAL RATIO

In [ ]:
# Compute vocal to other sound ratio using harmonic and persurcive energy

In [ ]:
import librosa
import numpy as np

In [ ]:
def compute_vocals_to_other_ratio(audio_path):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Apply Harmonic/Percussive Source Separation
    harmonic, percussive = librosa.effects.hpss(y)

    # Calculate the energy of the harmonic and percussive components
    harmonic_energy = np.sum(harmonic**2)
    percussive_energy = np.sum(percussive**2)

    # Calculate the total energy (harmonic + percussive)
    total_energy = harmonic_energy + percussive_energy

    # Calculate the ratio of harmonic energy (likely vocals) to the total energy
    vocals_to_other_ratio = harmonic_energy / total_energy

    return vocals_to_other_ratio


In [ ]:
model_dirs = [
    "/content/drive/MyDrive/audio_stemming/Outputs/Demucs_Output",
    "/content/drive/MyDrive/audio_stemming/Outputs/OpenUnmix_Output",
    "/content/drive/MyDrive/audio_stemming/Outputs/Spleeter_Output",
    "/content/drive/MyDrive/audio_stemming/Outputs/WaveUNet_Output"
]

In [ ]:
# Dictionary to store the ratios for each model
model_names = ["Demucs", "OpenUnmix", "Spleeter", "Wave"]
model_ratios = {model: [] for model in model_names}

In [ ]:
# Loop through each model and calculate the vocals-to-other ratio for each file
for model_name, model_dir in zip(model_names, model_dirs):
    for i in range(1, 37):  # 36 audio files
        audio_file = f"{model_name}_vocal_{i}.wav"
        audio_path = os.path.join(model_dir, audio_file)
        try:
            ratio = compute_vocals_to_other_ratio(audio_path)
            model_ratios[model_name].append(ratio)
        except Exception as e:
            print(f"Error processing {audio_path}: {e}")

In [ ]:
# Print the results for each model
for model, ratios in model_ratios.items():
    avg_ratio = np.mean(ratios) if ratios else 0
    print(f"{model} - Average Vocals-to-Other Ratio: {avg_ratio:.4f}")

Demucs - Average Vocals-to-Other Ratio: 0.8127
OpenUnmix - Average Vocals-to-Other Ratio: 0.8162
Spleeter - Average Vocals-to-Other Ratio: 0.8259
Wave - Average Vocals-to-Other Ratio: 0.7665
